<style type="text/css">.tg  {border-collapse:collapse;border-spacing:0;}.tg td{border-color:rgb(16, 137, 182);border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}.tg th{border-color:rgb(16, 137, 182);border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}.tg .tg-73oq{border-color:rgb(10, 89, 162);text-align:left;vertical-align:middle}.tg .tg-42lt{border-color:rgb(10, 89, 162);text-align:center;vertical-align:middle}.tg .tg-5qt9{font-size:small;text-align:left;vertical-align:middle}</style><table class="tg"><thead>  <tr>    <th class="tg-73oq"><img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="Argopy logo" width="120" height="60"></th>    <th class='tg-42lt'><h1>Argopy plotters</h1></th>  </tr></thead><tbody>  <tr>    <td class="tg-5qt9" colspan="2"><span style="font-weight:bold">Author :</span> <a href="//annuaire.ifremer.fr/cv/17182" target="_blank" rel="noopener noreferrer">G. Maze</a></td>  </tr>  <tr>    <td class="tg-5qt9" colspan="2">🏷️ This notebook is compatible with Argopy versions &gt;= <a href="https://argopy.readthedocs.io/en/1.3.1" target="_blank" rel="noopener noreferrer">1.3.1</a></td>  </tr>  <tr>    <td class="tg-5qt9" colspan="2">© <a href="https://github.com/euroargodev/argopy-training/blob/main/LICENSE" target="_blank" rel="noopener noreferrer">European Union Public Licence (EUPL) v1.2</a>, see at the bottom of this notebook for more.</td>  </tr></tbody></table>
**Description:**

This notebook describes low-level plotting features available in Argopy:
- [scatter maps (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.plot.scatter_map.html#argopy.plot.scatter_map)
- [scatter plots (although this function is still in experimental mode) (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.plot.scatter_plot.html)

These functions can consume Xarray Dataset as well as Pandas Dataframe, as long as such structures have been fetched/loaded from Argopy.

This notebook also covers the [Argopy utility class for colors (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.ArgoColors.html).

This is an extensive exploration of ["Data visualisation - utilities" section of the documentation (↗)](https://argopy.readthedocs.io/en/1.3.1/user-guide/working-with-argo-data/visualisation.html#utilities).


**Table of Contents**
- [Load some data with a DataFetcher](#load-some-data-with-a-datafetcher)
- [Scatter maps](#scatter-maps)
    - [🛟 NOTE](#🛟-note)
    - [✏️ EXERCISE](#✏️-exercise)
- [Use any colormap](#use-any-colormap)
- [Scatter plots](#scatter-plots)
***

## Load some data with a DataFetcher

Let's start with the basic import:

In [ ]:
# Shut down some warning messages for clarity
import warnings
warnings.filterwarnings("ignore")

# Import the Argopy features we want to work with in this notebook:
from argopy.plot import scatter_map, scatter_plot, ArgoColors
from argopy import DataFetcher, ArgoFloat, ArgoIndex

and load some Argo-core data off the coast of Japan to work with:

In [ ]:
%%time
fetcher = DataFetcher(src='erddap', cache=True, parallel=True).region([130, 160, 28, 38, 0, 1000, '2010','2011']).load()
ds_region = fetcher.data
ds_region.argo

## Scatter maps

With [scatter_map (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.plot.scatter_map.html#argopy.plot.scatter_map), each point is an Argo profile location, colored with a user defined variable and colormap. 

By default, floats trajectories are always shown, but this can be changed with the ``traj=False`` boolean argument.

#### 🛟 NOTE
When using this function with a xarray Dataset, it must be in the form of a [collection of profiles (↗)](https://argopy.readthedocs.io/en/1.3.1/user-guide/working-with-argo-data/data_manipulation.html#points-vs-profiles).  
Moreover, by default, if more than one N_LEVELS is found in the dataset, [scatter_map (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.plot.scatter_map.html#argopy.plot.scatter_map) will use the first level only.


So we first need to transform the collection of points into a collection of profiles:

In [ ]:
dsp_region = ds_region.argo.point2profile()
dsp_region.argo

<br>

By default, there will be one color attributed to one float:

In [ ]:
fig, ax, hdl = scatter_map(dsp_region, legend=False); # we hide the legend because it is too large with so many floats

<br>

🔍 **The variable to use for color coding is set with the ``hue`` argument**.

The colormap is finely tuned for some variables (QC flags, Data Mode and Deployment Status)

In [ ]:
fig, ax, hdl = scatter_map(dsp_region, hue='TEMP_QC')

In [ ]:
fig, ax, hdl = scatter_map(dsp_region, hue='DATA_MODE')

<br>

Since the function can easily consume Pandas Dataframe, it can be called from an [ArgoIndex (↗)](https://argopy.readthedocs.io/en/lastest/advanced-tools/stores/argoindex.html) export:

In [ ]:
idx = ArgoIndex(cache=True).query.wmo([6902771, 4903348])
df = idx.to_dataframe()
df

In [ ]:
scatter_map(df, padding=2, markersize=12);

#### ✏️ EXERCISE

Load data for a given float, as an [ArgoFloat (↗)](https://argopy.readthedocs.io/en/1.3.1/advanced-tools/stores/argofloat.html) instance to get more variables, and plot a map of the  `PROFILE_PSAL_QC` parameter.

In [ ]:
# Your code

## Use any colormap

Beyond the predefined set of Argo colors used above, one can use any colormap that can be discretesized.

In the example below, we plot profile years of sampling using some colormap:

In [ ]:
df

In [ ]:
dsp_float['year'] = dsp_float['JULD.year']  # Add new variable to the dataset
scatter_map(dsp_float,
            hue='year',
            cmap='Set1',
            legend_title='Year of sampling');

## Scatter plots

We now illustrate the **experimental** [scatter_plot (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.plot.scatter_plot.html#argopy.plot.scatter_plot) function.

This function consumes xarray Dataset with a collection of points or profiles.

Note that this function is an *Argo-informed* layer on top of the [pyplot scatter function (↗)](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html).

In [ ]:
fig, ax, m = scatter_plot(ds_region, 'TEMP');

In [ ]:
fig, ax, m, cb = scatter_plot(ds_region, 'PSAL_QC', cbar=True, vmin=0, vmax=9+1);

# Argo Colors


With the [ArgoColors class (↗)](https://argopy.readthedocs.io/en/1.3.1/generated/argopy.ArgoColors.html), Argopy provides [documented discretized colormaps (↗)](https://argopy.readthedocs.io/en/1.3.1/user-guide/working-with-argo-data/visualisation.html#argo-colors) to be re-used in your procedures. 

<br>
The list of documented colormaps is obtained like this:

In [ ]:
ArgoColors().list_valid_known_colormaps

<br>

A friendly representation of a colormap is return with the instance:

In [ ]:
ArgoColors('qc_flag')

<br>

and one can get colormap attributes with the ``known_colormaps`` dictionary:

In [ ]:
ArgoColors('qc_flag').definition

<br> 

To get the colormap as a [Matplotlib LinearSegmentedColormap (↗)](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.LinearSegmentedColormap.html#matplotlib.colors.LinearSegmentedColormap) to use in your plot, use the `cmap` property:

In [ ]:
ArgoColors('qc_flag').cmap


## 🏁 End of the notebook

***
#### 👀 Useful argopy commands
```python
argopy.reset_options()
argopy.show_options()
argopy.status()
argopy.clear_cache()
argopy.show_versions()
```
#### ⚖️ License Information
This Jupyter Notebook is licensed under the **European Union Public Licence (EUPL) v1.2**.

| Permissions      | Limitations     | Conditions                     |
|------------------|-----------------|--------------------------------|
| ✔ Commercial use | ❌ Liability     | ⓘ License and copyright notice |
| ✔ Modification   | ❌ Trademark use | ⓘ Disclose source              |
| ✔ Distribution   | ❌ Warranty      | ⓘ State changes                |
| ✔ Patent use     |                  | ⓘ Network use is distribution  |
| ✔ Private use    |                  | ⓘ Same license                 |

For more details, visit: [EUPL v1.2 Full Text (↗)](https://github.com/euroargodev/argopy-training/blob/main/LICENSE).

#### 🤝 Sponsor
![logo (↗)](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/for_nb_producers/disclaimer_argopy_EAONE.png)
***
